In [1]:
import platform

arch = platform.machine()
sys = platform.system()
processor = platform.processor()
print(f"{arch}\n{sys}\n{processor}")

ARM64
Windows
ARMv8 (64-bit) Family 8 Model 1 Revision 201, Qualcomm Technologies Inc


In [2]:
import onnxruntime as ort
import os
import numpy as np

from pathlib import Path
from tokenizers import Tokenizer

In [3]:
root_dir = Path.cwd().parent.parent
root_dir

WindowsPath('C:/Users/DFS/Desktop/gitrepo/qnn_sample_apps')

In [4]:
onnx_root = Path(ort.__file__).parent
onnx_root

WindowsPath('C:/Users/DFS/Desktop/gitrepo/env_arm64/Lib/site-packages/onnxruntime')

In [5]:
model_subdirectory = "DeepSeek-R1-Distilled-NPU-Optimized"

# The embeddings model is entry point, use netron to visualize
model_name = "deepseek_r1_1_5_embeddings_quant_v2.0.onnx"

# Need this for "context" graph (processes first hidden states)
context_model = "deepseek_r1_1_5_ctx_v2.2.onnx_ctx.onnx"

# Processes second hidden states before passing to head
context_model_iter = "deepseek_r1_1_5_iter_v2.2.onnx_ctx.onnx"

# Initial stage just provides hidden states, need to get logits by applying model head.
head_model = "deepseek_r1_1_5_head_quant_v2.0.onnx"

# Genai configuration path
configuration_json = "genai_config.json"

# Tokenizer
tokenizer_json = "tokenizer.json"

In [6]:
model_path = root_dir/"models"/model_subdirectory/model_name
ctx_path = root_dir/"models"/model_subdirectory/context_model
ctx_path_itr = root_dir/"models"/model_subdirectory/context_model_iter
head_path = root_dir/"models"/model_subdirectory/head_model
tokenizer_path = root_dir/"models"/model_subdirectory/tokenizer_json
config_path = root_dir/"models"/model_subdirectory/configuration_json
hexagon_driver = onnx_root/"capi"/"QnnHtp.dll"

In [7]:
session_options = ort.SessionOptions()

session_options.add_session_config_entry("ep.context_enable","0")
session_options.add_session_config_entry("ep.context_file_path",str(ctx_path))
session_options.add_session_config_entry("ep.context_embed_mode","1")
session_options.add_session_config_entry("qnn.backend_config_path",str(config_path))

qnn_provider_options = {
    "backend_path": hexagon_driver
}

embedding_session = ort.InferenceSession(model_path,
                                providers= [("QNNExecutionProvider",qnn_provider_options)],
                               sess_options= session_options
                              )

head_session = ort.InferenceSession(head_path,
                                providers= [("QNNExecutionProvider",qnn_provider_options)],
                               sess_options= session_options
                              )

ctx_session = ort.InferenceSession(ctx_path,
                                    providers=[("QNNExecutionProvider",qnn_provider_options)],
                                    sess_options= session_options
                                        )

ctx_itr_session = ort.InferenceSession(ctx_path_itr,
                                         providers=[("QNNExecutionProvider",qnn_provider_options)],
                                         sess_options= session_options
                                      )
embedding_session.get_providers()

['QNNExecutionProvider', 'CPUExecutionProvider']

In [8]:
inputs = embedding_session.get_inputs()
outputs = embedding_session.get_outputs()
input_0 = inputs[0]
output_0 = outputs[0]

In [9]:
print(f"Expected Input Shape: {input_0.shape}")
print(f"Expected Input Type: {input_0.type}")
print(f"Expected Input Name: {input_0.name}")

Expected Input Shape: [1, 'seq_len']
Expected Input Type: tensor(int64)
Expected Input Name: input_ids


In [10]:
print(f"Expected Output Shape: {output_0.shape}")
print(f"Expected Output Type: {output_0.type}")
print(f"Expected Output Name: {output_0.name}")

Expected Output Shape: [1, 'seq_len', 1536]
Expected Output Type: tensor(float)
Expected Output Name: input_hidden_states


In [11]:
inputs_ctx = ctx_session.get_inputs()
outputs_ctx = ctx_session.get_outputs()
input_0_ctx = inputs_ctx[0]
output_0_ctx = outputs_ctx[0]

In [12]:
print(f"Expected Input Shape: {input_0_ctx.shape}")
print(f"Expected Input Type: {input_0_ctx.type}")
print(f"Expected Input Name: {input_0_ctx.name}")

Expected Input Shape: [1, 2, 'max_seq_len', 128]
Expected Input Type: tensor(float)
Expected Input Name: past_keys_0


In [13]:
print(f"Expected Output Shape: {output_0_ctx.shape}")
print(f"Expected Output Type: {output_0_ctx.type}")
print(f"Expected Output Name: {output_0_ctx.name}")

Expected Output Shape: [1, 64, 1536]
Expected Output Type: tensor(float)
Expected Output Name: output_hidden_states


In [14]:
inputs_ctx_itr = ctx_itr_session.get_inputs()
outputs_ctx_itr = ctx_itr_session.get_outputs()
input_0_ctx_itr = inputs_ctx_itr[0]
output_0_ctx_itr = outputs_ctx_itr[0]

In [15]:
print(f"Expected Input Shape: {input_0_ctx_itr.shape}")
print(f"Expected Input Type: {input_0_ctx_itr.type}")
print(f"Expected Input Name: {input_0_ctx_itr.name}")

Expected Input Shape: [1, 1, 1536]
Expected Input Type: tensor(float)
Expected Input Name: input_hidden_states


In [16]:
print(f"Expected Output Shape: {output_0_ctx_itr.shape}")
print(f"Expected Output Type: {output_0_ctx_itr.type}")
print(f"Expected Output Name: {output_0_ctx_itr.name}")

Expected Output Shape: [1, 2, 'max_seq_len', 128]
Expected Output Type: tensor(float)
Expected Output Name: present_keys_0


In [17]:
inputs_head = head_session.get_inputs()
outputs_head = head_session.get_outputs()
input_0_head = inputs_head[0]
output_0_head = outputs_head[0]

In [18]:
print(f"Expected Input Name: {input_0_head.name}")
print(f"Expected Input Shape: {input_0_head.shape}")
print(f"Expected Input Type: {input_0_head.type}")

Expected Input Name: output_hidden_states
Expected Input Shape: [1, 'seq_len', 1536]
Expected Input Type: tensor(float)


In [19]:
print(f"Expected Output Name: {output_0_head.name}")
print(f"Expected Output Shape: {output_0_head.shape}")
print(f"Expected Output Type: {output_0_head.type}")

Expected Output Name: logits
Expected Output Shape: [1, 'seq_len', 151936]
Expected Output Type: tensor(float)


In [20]:
tokenizer = Tokenizer.from_file(str(tokenizer_path))

In [21]:
my_query = "<｜User｜>\nWhat is it like to be a dog? Please explain step by step.\n<｜Assistant｜> <think>\n"
encoding = tokenizer.encode(my_query)

In [22]:
print("Token IDs:", encoding.ids)
print("Tokens:", encoding.tokens)

Token IDs: [151646, 151644, 198, 3838, 374, 432, 1075, 311, 387, 264, 5562, 30, 5209, 10339, 3019, 553, 3019, 624, 151645, 220, 151648, 198]
Tokens: ['<｜begin▁of▁sentence｜>', '<｜User｜>', 'Ċ', 'What', 'Ġis', 'Ġit', 'Ġlike', 'Ġto', 'Ġbe', 'Ġa', 'Ġdog', '?', 'ĠPlease', 'Ġexplain', 'Ġstep', 'Ġby', 'Ġstep', '.Ċ', '<｜Assistant｜>', 'Ġ', '<think>', 'Ċ']


In [23]:
input_ids = encoding.ids
input_ids

[151646,
 151644,
 198,
 3838,
 374,
 432,
 1075,
 311,
 387,
 264,
 5562,
 30,
 5209,
 10339,
 3019,
 553,
 3019,
 624,
 151645,
 220,
 151648,
 198]

In [24]:
# pad_token_id = tokenizer.token_to_id("<|pad|>") or 0
# pad_token_id

In [25]:
# pad the inputs to expected size of seq_len of 64
# target_seq_len = 64
# input_ids += [pad_token_id] * (target_seq_len - len(input_ids))
input_ids = np.array([input_ids], dtype=np.int64)
input_ids.shape

(1, 22)

In [26]:
# Run embedding session first
embedding_output = embedding_session.run(None, {"input_ids":input_ids})[0]
embedding_output.shape

(1, 22, 1536)

In [27]:
embedding_output

array([[[-0.04297549,  0.        ,  0.00537194, ...,  0.01911739,
         -0.01274492,  0.01911739],
        [-0.04292785,  0.00536598,  0.01073196, ...,  0.02076933,
         -0.01384622,  0.01384622],
        [ 0.01857128, -0.00928564, -0.03714256, ...,  0.01259027,
          0.02518054, -0.05036108],
        ...,
        [ 0.04204254, -0.        , -0.05605672, ...,  0.01426021,
         -0.04278062, -0.01426021],
        [-0.04041443,  0.        ,  0.0050518 , ...,  0.01299553,
         -0.01299553,  0.01949329],
        [ 0.01857128, -0.00928564, -0.03714256, ...,  0.01259027,
          0.02518054, -0.05036108]]], dtype=float32)

In [28]:
# Preparing inputs for prompt
batch_size = 1
seq_len = embedding_output.shape[1]
hidden_size = embedding_output.shape[2]
num_heads = 2
attn_head_size = 128 #hidden_size // num_heads
num_layers = 28
max_seq_len = 64

In [29]:
attn_head_size

128

In [30]:
hidden_size

1536

In [31]:
empty_kv = {}
for i in range(num_layers):
    past_shape = (batch_size, num_heads, max_seq_len, attn_head_size)
    empty_kv[f"past_keys_{i}"] = np.zeros(past_shape, dtype=np.float32)
    empty_kv[f"past_values_{i}"] = np.zeros(past_shape, dtype=np.float32)

len(empty_kv)

56

In [32]:
empty_kv.keys()

dict_keys(['past_keys_0', 'past_values_0', 'past_keys_1', 'past_values_1', 'past_keys_2', 'past_values_2', 'past_keys_3', 'past_values_3', 'past_keys_4', 'past_values_4', 'past_keys_5', 'past_values_5', 'past_keys_6', 'past_values_6', 'past_keys_7', 'past_values_7', 'past_keys_8', 'past_values_8', 'past_keys_9', 'past_values_9', 'past_keys_10', 'past_values_10', 'past_keys_11', 'past_values_11', 'past_keys_12', 'past_values_12', 'past_keys_13', 'past_values_13', 'past_keys_14', 'past_values_14', 'past_keys_15', 'past_values_15', 'past_keys_16', 'past_values_16', 'past_keys_17', 'past_values_17', 'past_keys_18', 'past_values_18', 'past_keys_19', 'past_values_19', 'past_keys_20', 'past_values_20', 'past_keys_21', 'past_values_21', 'past_keys_22', 'past_values_22', 'past_keys_23', 'past_values_23', 'past_keys_24', 'past_values_24', 'past_keys_25', 'past_values_25', 'past_keys_26', 'past_values_26', 'past_keys_27', 'past_values_27'])

In [33]:
empty_kv.values()

dict_values([array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32), array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0

In [34]:
embedding_output.shape

(1, 22, 1536)

In [35]:
# subtract 1 off shape to account for indexing from 0-15
init_sequence_length = np.array(embedding_output.shape[1]-1, dtype=np.int32).reshape(1,1)
max_seq_length = np.array([max_seq_len], dtype=np.int32)

In [36]:
init_sequence_length = np.array(5, dtype=np.int32).reshape(1,1)

seq_lens = {
    "past_seq_len": init_sequence_length,
    "total_seq_len": max_seq_length #seq_len
}
seq_lens

{'past_seq_len': array([[5]]), 'total_seq_len': array([64])}

In [37]:
embedding_output.shape

(1, 22, 1536)

In [38]:
# pad the inputs to expected size of seq_len of 64
batch_size, seq_len, embed_dim = embedding_output.shape
target_seq_len = 64

padded_embedding = np.zeros((batch_size, target_seq_len, embed_dim), dtype=embedding_output.dtype)

padded_embedding[:, :seq_len, :] = embedding_output
padded_embedding.shape

(1, 64, 1536)

In [39]:
# Check to ensure null vectors were added
padded_embedding[0,:18,:]

array([[-0.04297549,  0.        ,  0.00537194, ...,  0.01911739,
        -0.01274492,  0.01911739],
       [-0.04292785,  0.00536598,  0.01073196, ...,  0.02076933,
        -0.01384622,  0.01384622],
       [ 0.01857128, -0.00928564, -0.03714256, ...,  0.01259027,
         0.02518054, -0.05036108],
       ...,
       [ 0.03292911,  0.03292911, -0.01646456, ..., -0.01753568,
         0.02630352, -0.0438392 ],
       [ 0.        ,  0.01480107,  0.00740054, ...,  0.        ,
         0.0364337 , -0.01457348],
       [ 0.        , -0.02297287, -0.00765762, ...,  0.        ,
         0.01667006, -0.05001017]], dtype=float32)

In [40]:
init_prompt_inputs = {
    "input_hidden_states": padded_embedding, #embedding_output,
    **empty_kv,
    **seq_lens
}
init_prompt_inputs

{'input_hidden_states': array([[[-0.04297549,  0.        ,  0.00537194, ...,  0.01911739,
          -0.01274492,  0.01911739],
         [-0.04292785,  0.00536598,  0.01073196, ...,  0.02076933,
          -0.01384622,  0.01384622],
         [ 0.01857128, -0.00928564, -0.03714256, ...,  0.01259027,
           0.02518054, -0.05036108],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]]], dtype=float32),
 'past_keys_0': array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
 

In [41]:
init_prompt_inputs.keys()

dict_keys(['input_hidden_states', 'past_keys_0', 'past_values_0', 'past_keys_1', 'past_values_1', 'past_keys_2', 'past_values_2', 'past_keys_3', 'past_values_3', 'past_keys_4', 'past_values_4', 'past_keys_5', 'past_values_5', 'past_keys_6', 'past_values_6', 'past_keys_7', 'past_values_7', 'past_keys_8', 'past_values_8', 'past_keys_9', 'past_values_9', 'past_keys_10', 'past_values_10', 'past_keys_11', 'past_values_11', 'past_keys_12', 'past_values_12', 'past_keys_13', 'past_values_13', 'past_keys_14', 'past_values_14', 'past_keys_15', 'past_values_15', 'past_keys_16', 'past_values_16', 'past_keys_17', 'past_values_17', 'past_keys_18', 'past_values_18', 'past_keys_19', 'past_values_19', 'past_keys_20', 'past_values_20', 'past_keys_21', 'past_values_21', 'past_keys_22', 'past_values_22', 'past_keys_23', 'past_values_23', 'past_keys_24', 'past_values_24', 'past_keys_25', 'past_values_25', 'past_keys_26', 'past_values_26', 'past_keys_27', 'past_values_27', 'past_seq_len', 'total_seq_len'])

In [42]:
prompt_outputs = ctx_session.run(None, init_prompt_inputs)
len(prompt_outputs)

57

In [43]:
prompt_outputs[0].shape

(1, 64, 1536)

In [44]:
# Extract final hidden states and present_keys/values
print("Batch, prompt length (up to max 64 tokens), embedding size")
output_hidden_states = prompt_outputs[0]
output_hidden_states.shape

Batch, prompt length (up to max 64 tokens), embedding size


(1, 64, 1536)

In [45]:
print("Batch, key/value heads, prompt length (up to max 64 tokens), head dimension (size of projection for each head)")
print("Note: Total embedding size is 1536, this is split amongst 12 attention heads")
prompt_outputs[1].shape

Batch, key/value heads, prompt length (up to max 64 tokens), head dimension (size of projection for each head)
Note: Total embedding size is 1536, this is split amongst 12 attention heads


(1, 2, 64, 128)

In [46]:
prompt_outputs[1][0].shape

(2, 64, 128)

In [47]:
print("Prompt Length x Head Dimension (Embedding Window)")
prompt_outputs[1][0][0].shape

Prompt Length x Head Dimension (Embedding Window)


(64, 128)

In [48]:
# Populate initial past key/values
# Must start with index==1 because index==0 is output_hidden_states (see genai_config.json)
present_kv = {f"past_keys_{i}": prompt_outputs[1 + i * 2] for i in range(num_layers)}
present_kv.update({f"past_values_{i}": prompt_outputs[1 + i * 2 + 1] for i in range(num_layers)})
present_kv

{'past_keys_0': array([[[[ 8.30303383e+00, -1.78891516e+00, -1.94930065e+00, ...,
           -2.56777161e+02,  3.16983429e+02,  3.38252991e+02],
          [ 7.80517292e+00, -2.27671337e+00,  2.26585984e+00, ...,
           -2.56588531e+02,  3.16861755e+02,  3.38084015e+02],
          [ 1.30928874e+00,  1.94148183e+00,  1.50270534e+00, ...,
           -2.54622238e+02,  3.15024139e+02,  3.37630280e+02],
          ...,
          [-3.78288221e+00, -3.20166111e-01,  3.48864245e+00, ...,
           -2.51726898e+02,  3.13057556e+02,  3.35194275e+02],
          [ 9.41881418e-01, -1.06640840e+00,  3.07069492e+00, ...,
           -2.51673233e+02,  3.13009216e+02,  3.35148041e+02],
          [ 4.80068350e+00, -1.06169879e+00,  1.00537777e+00, ...,
           -2.51619583e+02,  3.12960876e+02,  3.35101807e+02]],
 
         [[ 3.52848077e+00,  2.82525206e+00,  3.41744471e+00, ...,
           -2.24700073e+02,  3.87306305e+02,  3.93166534e+02],
          [ 4.18862724e+00,  3.55295849e+00,  8.91697764e

In [49]:
present_kv.keys()

dict_keys(['past_keys_0', 'past_keys_1', 'past_keys_2', 'past_keys_3', 'past_keys_4', 'past_keys_5', 'past_keys_6', 'past_keys_7', 'past_keys_8', 'past_keys_9', 'past_keys_10', 'past_keys_11', 'past_keys_12', 'past_keys_13', 'past_keys_14', 'past_keys_15', 'past_keys_16', 'past_keys_17', 'past_keys_18', 'past_keys_19', 'past_keys_20', 'past_keys_21', 'past_keys_22', 'past_keys_23', 'past_keys_24', 'past_keys_25', 'past_keys_26', 'past_keys_27', 'past_values_0', 'past_values_1', 'past_values_2', 'past_values_3', 'past_values_4', 'past_values_5', 'past_values_6', 'past_values_7', 'past_values_8', 'past_values_9', 'past_values_10', 'past_values_11', 'past_values_12', 'past_values_13', 'past_values_14', 'past_values_15', 'past_values_16', 'past_values_17', 'past_values_18', 'past_values_19', 'past_values_20', 'past_values_21', 'past_values_22', 'past_values_23', 'past_values_24', 'past_values_25', 'past_values_26', 'past_values_27'])

In [50]:
# Dimension checks
present_kv["past_keys_0"].shape

(1, 2, 64, 128)

In [51]:
present_kv["past_keys_27"].shape

(1, 2, 64, 128)

In [52]:
output_hidden_states.shape

(1, 64, 1536)

In [53]:
logits = head_session.run(None, {"output_hidden_states": output_hidden_states})[0]
logits

array([[[ 4.112325  ,  2.9755692 ,  4.669573  , ..., -5.0415926 ,
         -5.03799   , -5.0237675 ],
        [ 4.0822787 ,  2.8075435 ,  4.5837374 , ..., -4.947254  ,
         -4.9431953 , -4.9273205 ],
        [ 5.303215  ,  7.5436454 ,  4.8844867 , ..., -0.07982719,
         -0.08396041, -0.07304609],
        ...,
        [ 9.087334  ,  5.828527  ,  3.2812564 , ...,  1.9324964 ,
          1.9283237 ,  1.9128788 ],
        [ 9.554918  ,  7.1117787 ,  5.7879353 , ...,  3.37149   ,
          3.366538  ,  3.35865   ],
        [ 7.706373  ,  6.2081037 ,  4.2435203 , ...,  1.4953384 ,
          1.4832134 ,  1.5056763 ]]], dtype=float32)

In [54]:
logits.shape

(1, 64, 151936)

In [55]:
logits[0,-1].shape

(151936,)

In [56]:
# Greedy Inference
# Grabs last tokens logits
next_token_id = int(np.argmax(logits[0, -1]))
next_token_id

594

In [57]:
tokenizer.decode([next_token_id])

"'s"

In [58]:
max_tokens = 50

generated_ids = [next_token_id]
prev_seq_len = 64

for _ in range(max_tokens):
    input_ids = np.array([[next_token_id]], dtype=np.int64)
    # print(tokenizer.decode(generated_ids, skip_special_tokens=True))
    # print(tokenizer.decode([next_token_id], skip_special_tokens=True))
    embedding_output = embedding_session.run(None, {"input_ids": input_ids})[0]

    # print(embedding_output.shape)

    lengths = {
    "past_seq_len": np.array([[prev_seq_len]], dtype=np.int32),
    "total_seq_len": np.array([prev_seq_len + 1], dtype=np.int32)
    }

    iter_inputs = {
    **present_kv,
    "input_hidden_states": embedding_output,
    **lengths,
    }

    iter_outputs = ctx_itr_session.run(None, iter_inputs)

    # Hidden states are stored in last index of iter outputs
    output_hidden_states = iter_outputs[-1]
    
    # For output tensor update key/value layers start at index = 0 
    present_kv = {f"past_keys_{i}": iter_outputs[i * 2] for i in range(num_layers)}
    present_kv.update({f"past_values_{i}":iter_outputs[i * 2 + 1] for i in range(num_layers)})
    logits = head_session.run(None, {"output_hidden_states": output_hidden_states})[0]

    next_token_id = int(np.argmax(logits[0, -1]))
    generated_ids.append(next_token_id)

    prev_seq_len += 1

    if next_token_id == tokenizer.token_to_id("< | end_of_sentence | >"):
        break

output_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
print("")
print("*"*100)
print("\nInitial Query:\n", my_query)
print("Generated:", output_text)

's
omething
 is
 it
 is



The
 question
 is
 asking
 about



The
 question
 is
 asking
 about



The
 question
 is
 asking
 about



The
 question
 is
 asking
 about



The
 question
 is
 asking
 about



The
 question
 is
 asking
 about



The
 question
 is
 asking
 about



The
 question

****************************************************************************************************

Initial Query:
 <｜User｜>
What is it like to be a dog? Please explain step by step.
<｜Assistant｜> <think>

Generated: 'something is it is

The question is asking about

The question is asking about

The question is asking about

The question is asking about

The question is asking about

The question is asking about

The question is asking about

The question is
